In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *

In [2]:
train = pd.read_csv('train.csv',index_col = 'Id')
model = setup(train,target = 'SalePrice')

 
Setup Succesfully Completed.


,Description,Value
0,session_id,781
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(1460, 80)"
4,Missing Values,True
5,Numeric Features,20
6,Categorical Features,59
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [3]:
compare_models(whitelist = ['rf','lightgbm','xgboost'],sort = 'RMSE',)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Extreme Gradient Boosting,17625.4225,911741867.5350,29212.0811,0.8604,0.1407,0.1011,0.5969
1,Light Gradient Boosting Machine,18272.3613,1003640834.2679,30461.7572,0.8466,0.1471,0.1038,0.2264
2,Random Forest,18676.3988,1088159967.5318,31973.5269,0.8303,0.1548,0.1088,1.8060


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=781,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=0)

In [4]:
xgb = create_model(estimator = 'xgboost',ensemble = True,method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16246.3579,546367507.0216,23374.5055,0.8647,0.1447,0.1088
1,14932.4226,469106048.6331,21658.8561,0.8969,0.1348,0.1000
2,18010.6086,697300805.1456,26406.4539,0.8776,0.1613,0.1154
3,15985.5803,655022714.2016,25593.4115,0.8954,0.1157,0.0869
4,19885.9220,1399016564.1356,37403.4298,0.8180,0.1564,0.1060
5,15917.5143,568487749.0096,23842.9811,0.9123,0.1235,0.0893
6,14681.6573,366996421.1841,19157.1507,0.9178,0.1042,0.0835
7,21833.5427,1993352911.8051,44646.9810,0.7991,0.1681,0.1103
8,19532.6477,1473409701.6786,38385.0192,0.7812,0.1496,0.1008
9,16598.3759,495151045.8747,22251.9897,0.9253,0.1374,0.1043


In [5]:
tuned_xgb = tune_model(estimator = xgb,optimize = 'RMSE',choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16092.7811,637199002.3244,25242.8010,0.8422,0.1473,0.1079
1,14914.5221,451944276.6069,21258.9811,0.9007,0.1328,0.0986
2,18234.3189,707150756.2586,26592.3063,0.8759,0.1634,0.1172
3,16463.7150,647032023.9527,25436.8242,0.8967,0.1217,0.0911
4,20214.2418,1455030021.7804,38144.8558,0.8108,0.1604,0.1081
5,15325.1277,504493405.3827,22460.9306,0.9222,0.1173,0.0846
6,13404.3554,332361723.6210,18230.7905,0.9256,0.0959,0.0753
7,20946.4746,1798537412.5500,42409.1666,0.8187,0.1622,0.1068
8,19320.7388,1198350841.8322,34617.2044,0.8221,0.1382,0.1004
9,17561.5133,550863433.8059,23470.4801,0.9169,0.1443,0.1108


In [6]:
test = pd.read_csv('test.csv',index_col = 'Id' )
result = test[['LotArea','Alley']]
result

,LotArea,Alley
Id,,
1461,11622,NaN
1462,14267,NaN
1463,13830,NaN
1464,9978,NaN
1465,5005,NaN
...,...,...
2915,1936,NaN
2916,1894,NaN
2917,20000,NaN


In [7]:
predictions = predict_model(tuned_xgb,data = test)
predictions['Label']

0       119803.085938
1       161999.578125
2       192777.140625
3       202734.062500
4       187014.406250
            ...      
1454     97851.179688
1455     88672.710938
1456    150938.937500
1457    137576.375000
1458    219649.453125
Name: Label, Length: 1459, dtype: float32

In [8]:
result['SalePrice'] = predictions['Label'].values
result.drop(['LotArea','Alley',],axis = 1,inplace = True)
result

,SalePrice
Id,
1461,119803.085938
1462,161999.578125
1463,192777.140625
1464,202734.062500
1465,187014.406250
...,...
2915,97851.179688
2916,88672.710938
2917,150938.937500


In [9]:
result.to_csv('pycaret_submission.csv')